### 01 packages

In [9]:
import os
import praw
import csv
import time
from datetime import datetime, timezone
import pandas as pd
from tqdm import tqdm
import openai
import pandas as pd
import time
import json

### 02 rule-based cleaning

In [12]:
###################################################################################################################
###################################################################################################################

output_dir = "02data"
input_file = os.path.join(output_dir, "user_reviews.csv")

###################################################################################################################
###################################################################################################################

df=pd.read_csv(input_file)

###################################################################################################################
###################################################################################################################

# ------------------- Preprocessing -------------------

# Create a new column 'description_clean' that:
# 1. Converts the 'description' column to string
# 2. Converts all text to lowercase (for case-insensitive matching)
df['description_clean'] = df['description'].astype(str).str.lower()

# Calculate the length of each description; could be used to filter very short (likely uninformative) texts.
df['description_length'] = df['description_clean'].str.len()

# ------------------- Filtering -------------------

# Define a set of keywords that are likely to indicate the review is related to a car purchase or review.
# You can expand or adjust this list depending on your needs.
filter_keywords = [
    "bought",              # common verb for purchase
    "purchased",           # alternative verb
    "got",                 # sometimes used casually
    "buy",                 # generic purchase verb (e.g., "buying a car")
    "car review",          # directly indicates a review
    "my new car",          # ownership confirmation
    "car purchase",        # indicates a purchase experience
    "review"               # generic review keyword (may add noise, adjust as needed)
]

# Define a helper function that checks whether a given text contains any of the keywords.
def contains_keyword(text, keywords):
    for kw in keywords:
        if kw in text:
            return True
    return False
    
# Apply the filtering function to create a boolean column 'relevant'
df['relevant'] = df['description_clean'].apply(lambda x: contains_keyword(x, filter_keywords))

# Select only the rows that are marked as relevant (i.e., contain at least one keyword)
filtered_df = df[df['relevant'] == True].copy()

# ------------------- Output -------------------

# Show the number of total reviews and how many passed the filter
print("Total reviews: ", len(df))
print("Filtered reviews: ", len(filtered_df))


Total reviews:  5923
Filtered reviews:  4258


### 03 LLM-based cleaning

In [ ]:
# ---------------------- Configuration ----------------------
with open("secret.txt", "r") as f:
    openai.api_key = f.read().strip()  # Read the ChatGPT API key

# ---------------------- Define LLM Analysis Function ----------------------

def remove_markdown_formatting(text):
    """
    Removes markdown code block markers from a string.
    If the text starts with ``` (possibly followed by a language tag)
    and ends with ```, remove those lines.
    """
    lines = text.strip().splitlines()
    if lines and lines[0].startswith("```"):
        # Remove the first line (opening marker) and the last line if it contains ```
        lines = lines[1:]
        if lines and lines[-1].strip().startswith("```"):
            lines = lines[:-1]
    return "\n".join(lines).strip()


def analyze_review(description):
    """
    Uses the OpenAI ChatCompletion API to analyze a car review description.
    The prompt instructs the model to carefully read the review text and extract:
      1. has_car: "Yes" if the review clearly states that the user bought or owns a car; "No" otherwise.
      2. car_model: The exact car model mentioned (e.g., "Tesla Model 3", "BMW 3 Series");
         return an empty string if none is clearly mentioned.
      3. fuel_type: "Electric" if the review indicates an electric vehicle,
         "Conventional" if it indicates a fuel-based vehicle, or "Unknown" if it cannot be determined.
    
    Args:
        description (str): The review text.
    
    Returns:
        dict: A dictionary with keys "has_car", "car_model", and "fuel_type" if successful;
              otherwise, returns None.
    """
    prompt = f"""
You are an expert automobile analyst. Carefully read the following review text and answer the following three questions:
1. Does the review clearly state that this user bought or owns a car? Answer "Yes" or "No".
2. If yes, what is the exact car model mentioned? (For example, "Tesla Model 3", "BMW 3 Series"). If none is clearly mentioned, return an empty string.
3. Based on the review text, determine whether the car is Electric or Conventional. Answer "Electric" if it is an electric vehicle, "Conventional" if it is a fuel-based vehicle, or "Unknown" if it cannot be determined.
Return your answer strictly in JSON format with exactly the keys: "has_car", "car_model", "fuel_type".

Review text:
\"\"\"{description}\"\"\"
"""
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",  # Change to "gpt-4" if available and desired.
            messages=[
                {"role": "system", "content": "You are an expert analyst."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=400,
            temperature=0.7,
        )
        raw_output = response.choices[0].message.content.strip()
        raw_output=remove_markdown_formatting(raw_output)
        # Parse the output as JSON
        result = json.loads(raw_output)
        return result
    except Exception as e:
        print(f"Error analyzing description: {e}")
        return None

# ---------------------- Process Each Filtered Review with Progress Bar ----------------------
results = []  # This list will hold the structured LLM analysis results.

# Use tqdm to display a progress bar during iteration.
for idx, row in tqdm(filtered_df.iterrows(), total=len(filtered_df), desc="Processing reviews"):
    description = row.get("description", "")
    user_id = row.get("user_id", "Unknown")
    
    analysis = analyze_review(description)
    if analysis:
        # Save the extracted fields and include original context fields.
        analysis["user_id"] = user_id
        analysis["comment_time"] = row.get("comment_time", "")
        results.append(analysis)
    else:
        print(f"Skipping row {idx} due to analysis error.")
    
    # Optional: Add a short delay to manage API rate limits.
    time.sleep(0.5)

# ---------------------- Save Final Results to CSV ----------------------
# Convert the results list into a DataFrame.
results_df = pd.DataFrame(results)
# Ensure that the output DataFrame contains the desired columns.
desired_cols = ["user_id", "comment_time", "has_car", "car_model", "fuel_type"]
results_df = results_df[[col for col in desired_cols if col in results_df.columns]]

output_file = os.path.join(output_dir, "filtered_user_reviews.csv")
results_df.to_csv(output_file, index=False)

print(f"LLM analysis complete. {len(results_df)} records saved to {output_file}")